### TensorFlow 설정하기

In [8]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__ )


TensorFlow version: 2.9.1


In [9]:
import cv2
from PIL import Image

### 데이터세트 로드하기
MNIST 데이터세트를 로드하고 준비합니다. 샘플 데이터를 정수에서 부동 소수점 숫자로 변환합니다.

In [10]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [11]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [12]:
Image.fromarray(x_train[0])


In [13]:
y_train[0]

5

In [14]:
x_train[0].shape

(28, 28)

In [15]:
x_train, x_test = x_train / 255.0, x_test/ 255.0

#### Dropout Image
<img src="https://i.stack.imgur.com/c2O5w.png">

In [16]:
# 1. 모델 생성
model = tf.keras.models.Sequential([
    #Flatten -> 2차원 numpy(이미지)를 1차원으로 펴주는 층
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    #과적합 방지를 위해서 노드의 20%를 제거
    tf.keras.layers.Dropout(0.2),
    #10개의 클래스(0~9)로 예측해야 함. 따라서 10개의 노드로 결과를 모아줌
    #softmax로 각 클래스의 확률을 반환...!
    tf.keras.layers.Dense(10, activation='softmax')
])

# 2. 컴파일(너 어떻게 학습할래?! 를 설정해주는 단계)
model.compile(
              #손실함수의 최솟값을 찾기 위한 step 설정
              optimizer = 'adam',
              #loss는 손실함수 지정
              loss = 'sparse_categorical_crossentropy',
              #훈련과정의 기준 지표 설정
              metrics=['accuracy'])

In [37]:
x_train.shape

(60000, 28, 28)

각 예시에서 모델은 각 클래스에 대해 하나씩, logits 또는 log-odds 스코어 벡터를 반환합니다.

In [17]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.12613587, 0.09301813, 0.04997853, 0.04843717, 0.10761485,
        0.15847613, 0.0709657 , 0.12411483, 0.08533566, 0.13592318]],
      dtype=float32)

tf.nn.softmax 함수는 다음과 같이 이러한 로짓을 각 클래스에 대한 확률로 변환합니다.

In [18]:
tf.nn.softmax(predictions).numpy()

array([[0.10258496, 0.09924322, 0.09506244, 0.09491604, 0.10070246,
        0.10595682, 0.09707862, 0.10237784, 0.0984837 , 0.10359392]],
      dtype=float32)

참고: tf.nn.softmax 함수를 네트워크의 마지막 레이어에 대한 활성화 함수로 베이킹할 수 있습니다.

In [19]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [20]:
loss_fn(y_train[:1], predictions).numpy()

2.2447236

In [21]:
model.compile(optimizer='adam', loss=loss_fn, metrics = ['accuracy'])

### 모델 훈련 및 평가하기
모델을 훈련하고 평가합니다.

In [22]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


c:\Users\pgho8\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 4s 2ms/step - loss: 0.2979 - accuracy: 0.9128
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1444 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1095 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0883 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0759 - accuracy: 0.9756


In [23]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0664 - accuracy: 0.9784
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0585 - accuracy: 0.9816
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0520 - accuracy: 0.9832
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0487 - accuracy: 0.9841
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0433 - accuracy: 0.9854


c:\Users\pgho8\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


313/313 - 1s - loss: 0.0694 - accuracy: 0.9802 - 710ms/epoch - 2ms/step


[0.06936967372894287, 0.9801999926567078]

In [24]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [25]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533688, 0.08533688, 0.08533689, 0.08533774, 0.08533688,
        0.0853369 , 0.08533688, 0.23196721, 0.08533688, 0.08533691],
       [0.08533786, 0.08534552, 0.23195156, 0.08533786, 0.08533786,
        0.08533786, 0.08533786, 0.08533786, 0.08533786, 0.08533786],
       [0.08533782, 0.23195238, 0.0853385 , 0.08533783, 0.08533946,
        0.08533782, 0.08533783, 0.0853427 , 0.08533789, 0.08533782],
       [0.23196413, 0.08533706, 0.08533706, 0.08533706, 0.08533706,
        0.08533706, 0.08533928, 0.08533706, 0.08533706, 0.08533706],
       [0.08534922, 0.08534913, 0.08534914, 0.08534913, 0.23177327,
        0.08534913, 0.08534914, 0.08534941, 0.08534913, 0.08543331]],
      dtype=float32)>